# Kmeans Algorithm

L'algorithme K-means répond au problème de partitionnement d'un ensemble de données en k groupes (ou clusters) basé sur les caractéristiques des données. Le but est de minimiser la variance intra-cluster et de maximiser la variance inter-cluster, c'est-à-dire que les données au sein d'un même cluster doivent être aussi similaires que possible, tandis que les données appartenant à différents clusters doivent être distinctes.

Cette premiere version sera une version sans numpy, pour tester apres avec potentiellement une version utilisant numpy pour constater la difference de performance.

Commençons par définir les étapes de l'algorithme K-means:

- Initialisation des centroïdes: Choisir k points aléatoires dans l'espace des données pour représenter les centroïdes des clusters.
- Assigner chaque point de données au centroïde le plus proche.
- Mettre à jour les centroïdes en calculant la moyenne des points de données de chaque cluster.
- Répéter les étapes 2 et 3 jusqu'à ce que les centroïdes ne changent plus.

Commençons par l'initialisation des centroids

In [ ]:
import math

def distance_euclidienne(point1, point2):
    return math.sqrt(sum((p1 - p2) ** 2 for p1, p2 in zip(point1, point2)))

#Test de la fonction
point1 = [1,2]
point2 = [3,4]

La formule pour calculer la probabilité de choisir un point x comme centroïde est la suivante:

$$
P(x) = \frac{D(x)^2}{\sum_{x \in X} D(x)^2}
$$

Fonction qui est appelé pour le choix du prochain centroid à initialisé

In [ ]:
import random

def choose_next_centroid(data, centroids):
    distances = []
    for point in data:
        min_distance = float('inf')
        for centroid in centroids:
            distance = distance_euclidienne(point, centroid)
            #print("distance entre ", point, " et ", centroid, " : ", distance)
            if distance < min_distance:
                min_distance = distance
        distances.append(min_distance)
    

    somme_des_carres = sum(distance**2 for distance in distances) #calcul de la somme des carrés des distances
    probabilites = [distance**2 / somme_des_carres for distance in distances] #calcul des probabilités


    index_next_centroid = random.choices(range(len(data)), weights=probabilites, k=1)[0] ## on effectue le choix du prochain centroid en fonction des probabilité
    return data[index_next_centroid]

Ensuite on a la fonction qui permet d'initialiser les centroids, ici nous utilisons l'initialisation kmeans++ qui est une amélioration de l'initialisation aléatoire.

In [ ]:
def kmeans_plusplus_initialization(data, k):
    index_first_centroid = [random.randint(0, len(data) - 1)] # de manière aléatoire
    centroids = [data[index_first_centroid[0]]]

    for i in range(1, k):
        # choix du prochain centroid
        next_centroid = choose_next_centroid(data, centroids)
        centroids.append(next_centroid)

    return centroids

#Test de la fonction
data = [[1,2], [3,4], [5,6], [7,8], [9,10]]
k = 2
centroids = kmeans_plusplus_initialization(data, k)

Fonction qui permet d'attribuer des points à des clusters basés sur la proximité des centroïdes

In [ ]:
def assign_points_to_clusters(data, centroids):
    assignments = []            # stock l'indice du centroide auquel chaque point est assigné
    for point in data:
        min_distance = float('inf')
        cluster_index = None
        for i, centroid in enumerate(centroids):
            distance = distance_euclidienne(point, centroid)
            if distance < min_distance:
                min_distance = distance
                cluster_index = i
        assignments.append(cluster_index)
    return assignments

assignments = assign_points_to_clusters(data, centroids)

Après avoir attribuer les points aux clusters, on recalcule les positions des centroides

In [ ]:
from collections import defaultdict

def update_centroids(data, assignments, k):
    cluster_points = defaultdict(list)      # initialise automatiquement toute nouvelle clé avec une liste vide.
    for assignment, point in zip(assignments, data):    # regroupe les points par cluster
        cluster_points[assignment].append(point)
    
    new_centroids = []
    for i in range(k):
        points = cluster_points[i]
        if points:  # si le cluster contient des points
            new_centroid = [sum(dim)/len(dim) for dim in zip(*points)]
        else:       # si k est très grand ou si la distribution des points est très inégale
            new_centroid = random.choice(data)
        new_centroids.append(new_centroid)
    return new_centroids

new_centroids = update_centroids(data, assignments, k)

Ensuite on a cette fonction qui teste si l'algorithme a convergé c'est à dire qui test si les centroids ont changé

In [ ]:
def converge(old_centroids, new_centroids, tolerance=1e-4): #tolérance utilisé dans le kmeans de sklearn
    for old, new in zip(old_centroids, new_centroids): #on recupére les anciens et nouveaux centroïdes
        distance = 0
        for o, n in zip(old, new):
            distance += (o - n) ** 2 #on calcule la distance entre les anciens et nouveaux centroïdes

        if distance > tolerance ** 2: #si la distance est supérieur à la tolérance
            return False
    return True

Enfin on peut rassembler toutes ces fonctions pour créer l'algorithme K-means

In [ ]:
def kmeans_without_numpy(data, k, max_iters = 100):
    centroids = kmeans_plusplus_initialization(data, k) #initialisation des centroïdes
    for i in range(max_iters):
        #print("Centroids à l'itération {} :\n".format(i), centroids)
        assignments = assign_points_to_clusters(data, centroids) #assignation des points aux clusters
        new_centroids = update_centroids(data, assignments, k) #mise à jour des centroïdes
        if converge(centroids, new_centroids): #si l'algo converge (les centroïdes ne bouge plus)
            print("Convergence atteinte après {} itérations".format(i))
            break
        centroids = new_centroids #si non on met à jour les centroïdes et on continue
    return assignments, centroids

Test de l'algorithme sur un jeu de données basique

In [ ]:
data = [[random.random() for _ in range(2)] for _ in range(100)]
k = 4
assignments, centroids = kmeans_without_numpy(data, k)

Afin de mieux modulariser et organiser les tests, nous allons instancier la classe KMeans_Without_Numpy

In [ ]:
from kmeans_without_numpy import KMeans_Without_Numpy
kmeans_without_numpy = KMeans_Without_Numpy(k=4)

On lance l'execution de l'algorithme

In [ ]:
assignments,centroids = kmeans_without_numpy.fit(data)

Visualisation de l'algorithme K-means avec matplotlib pour mieux voir le resultat

In [ ]:
import matplotlib.pyplot as plt

colors = ['r', 'g', 'b', 'm']  
plt.figure(figsize=(10, 6))

for idx, centroid in enumerate(centroids):
    cluster_points = [data[i] for i in range(len(data)) if assignments[i] == idx]
    cluster_points_x = [point[0] for point in cluster_points]
    cluster_points_y = [point[1] for point in cluster_points]
        
    plt.scatter(cluster_points_x, cluster_points_y, c=colors[idx], label=f'Cluster {idx}')
        
    plt.scatter(centroid[0], centroid[1], c=colors[idx], marker='x', s=200, linewidths=3)

plt.title('Visualisation des Clusters et des Centroids')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.legend()
plt.show()

Ici on test avec la version de sklearn pour comparer les résultats et se situer par rapport à la performance de notre algorithme

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

datanp = np.array(data)

kmeans_sklearn = KMeans(n_clusters=4, init='k-means++')
kmeans_sklearn.fit(datanp)

In [ ]:
centroids_sklearn = kmeans_sklearn.cluster_centers_
labels_sklearn = kmeans_sklearn.labels_

plt.figure(figsize=(10, 6))

for i in range(4):
    # Sélectionne les points de données du cluster i
    cluster_points = datanp[labels_sklearn == i]
    # Trace les points de données
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], c=colors[i], label=f'Cluster {i}')
    # Trace le centroid
    plt.scatter(centroids_sklearn[i, 0], centroids_sklearn[i, 1], c=colors[i], marker='x', s=200, linewidths=3)

plt.title('Clustering avec scikit-learn KMeans')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.legend()
plt.show()

On va commencé les tests avancés en utilisant des données générer par la fonction `make_blobs` de la bibliothèque `sklearn.datasets`. Ce sont des données artificielles qui sont habituellement employées pour tester des algorithmes de clustering . On va testé la qualité du clustering de notre version sans numpy avec la version Kmeans de sklearns, pour cela on va utilisé le score de silhouette.

Le score de silhouette est une mesure de la qualité d'un clustering qui évalue à quel point chaque point d'un cluster est similaire aux points de son propre cluster comparé à ceux des autres clusters. Ce score est particulièrement utile pour déterminer si le nombre de clusters utilisé dans l'analyse est approprié.

Pour chaque point `i`, le score de silhouette est calculé comme suit :

- a(i) : La distance moyenne entre le point `i` et tous les autres points dans le même cluster. Cela mesure à quel point le point `i` est bien intégré dans son cluster.

- b(i) : La distance moyenne minimale du point `i` à tous les points d'un autre cluster dont il n'est pas membre. Cela mesure à quel point le point `i` est loin des points du cluster voisin le plus proche.

Le score de silhouette pour un point individuel est alors donné par la formule :

$$
s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}
$$

- Le score de silhouette varie de −1 à +1.
- Un score élevé (proche de +1) indique que le point est bien placé à l'intérieur de son cluster et loin des autres clusters.

On va utiliser aussi la méthode du coude qui est une technique graphique utilisée pour aider à déterminer le nombre optimal de clusters dans un algorithme de clustering, comme KMeans. Cette méthode consiste à visualiser la variation de la somme des carrés des distances (SSE) entre les points et leurs centroids assignés, en fonction du nombre de clusters utilisés.

Cette fonction va gérer le calcul de la SSE et des scores de silhouette pour les versions customisées et Scikit-Learn de KMeans.

In [ ]:
def calculate_metrics(data, k_range):
    from kmeans_without_numpy import KMeans_Without_Numpy
    from sklearn.cluster import KMeans as SklearnKMeans
    from sklearn.metrics import silhouette_score
    
    sse_custom = {}
    sse_sklearn = {}
    silhouette_custom = {}
    silhouette_sklearn = {}
    
    for k in k_range:
        # Notre version de KMeans
        kmeans_custom = KMeans_Without_Numpy(k=k, max_iters=1000)
        _, centroids_custom = kmeans_custom.fit(data)
        sse_custom[k] = sum(min((sum((x - c) ** 2 for x, c in zip(point, centroid)) for centroid in centroids_custom)) for point in data)
        assignments_custom = [min(range(k), key=lambda i: sum((x - c) ** 2 for x, c in zip(point, centroids_custom[i]))) for point in data]
        silhouette_custom[k] = silhouette_score(data, assignments_custom)
        
        # Version scikit-learn de KMeans
        kmeans_sklearn = SklearnKMeans(n_clusters=k, init='k-means++', max_iter=1000, random_state=0)
        kmeans_sklearn.fit(data)
        sse_sklearn[k] = kmeans_sklearn.inertia_
        silhouette_sklearn[k] = silhouette_score(data, kmeans_sklearn.labels_)
    
    return sse_custom, sse_sklearn, silhouette_custom, silhouette_sklearn


Cette fonction gérera le tracé des graphiques pour les métriques calculées.

In [ ]:
def plot_metrics(sse_custom, sse_sklearn, silhouette_custom, silhouette_sklearn):

    plt.figure(figsize=(14, 7))
    # Ceci trace la méthode du coude pour les deux versions de KMeans
    plt.subplot(1, 2, 1)
    plt.plot(list(sse_custom.keys()), list(sse_custom.values()), 'bo-', label='Custom KMeans')
    plt.plot(list(sse_sklearn.keys()), list(sse_sklearn.values()), 'rx-', label='Scikit-Learn KMeans')
    plt.xlabel('Nombre de clusters')
    plt.ylabel('SSE')
    plt.title('Méthode du coude')
    plt.legend()

    # Ceci trace le score de silhouette pour les deux versions de KMeans
    plt.subplot(1, 2, 2)
    plt.plot(list(silhouette_custom.keys()), list(silhouette_custom.values()), 'bo-', label='Custom KMeans')
    plt.plot(list(silhouette_sklearn.keys()), list(silhouette_sklearn.values()), 'rx-', label='Scikit-Learn KMeans')
    plt.xlabel('Nombre de clusters')
    plt.ylabel('Score de Silhouette')
    plt.title('Score de Silhouette par nombre de clusters')
    plt.legend()

    plt.show()


In [ ]:
from sklearn.datasets import make_blobs

data, _ = make_blobs(n_samples=300, centers=4, cluster_std=0.60, random_state=0)

k_range = range(2, 10)
sse_custom, sse_sklearn, silhouette_custom, silhouette_sklearn = calculate_metrics(data, k_range)

plot_metrics(sse_custom, sse_sklearn, silhouette_custom, silhouette_sklearn)

Lors de la génaration de nos données, on a initialisé le paramètre centers à 4, ce qui signifie qu'on gènère nos 300 points autours de 4 centre distincts. On a appliqué KMeans avec `k=4` (où k est le nombre de clusters que l'algorithme doit trouver), c'est pour cela qu'on obtient autour de `k=4` les meilleurs résultats en termes de regroupement des données selon leur proximité aux centres générés initialement.

Les deux implémentations semblent offrir des performances similaires en termes de SSE et les scores de silhouette sont presque identiques, suggérant que malgré les différences potentielles en détails d'implémentation, les deux versions produisent des résultats de clustering de qualité similaire.

On va tester maintenant le temps d'execution en moyenne

In [ ]:
import time
import numpy as np
from sklearn.cluster import KMeans as SklearnKMeans
from kmeans_without_numpy import KMeans_Without_Numpy


def run_custom_kmeans(data):
    kmeans_custom = KMeans_Without_Numpy(k=4, max_iters=1000)
    start_time = time.time()
    assignments_custom, centroids_custom = kmeans_custom.fit(data)
    end_time = time.time()
    execution_time = end_time - start_time
    return centroids_custom, execution_time

def run_sklearn_kmeans(data):
    kmeans_sklearn = SklearnKMeans(n_clusters=4, init='k-means++',max_iter=1000, random_state=None)
    start_time = time.time()
    kmeans_sklearn.fit(data)
    end_time = time.time()
    execution_time = end_time - start_time
    return kmeans_sklearn.cluster_centers_, execution_time

def test_stability(data, runs=10):
    results_custom = []
    results_sklearn = []
    times_custom = []
    times_sklearn = []
    
    for _ in range(runs):
        centroids_custom, time_custom = run_custom_kmeans(data)
        centroids_sklearn, time_sklearn = run_sklearn_kmeans(data)
        results_custom.append(centroids_custom)
        results_sklearn.append(centroids_sklearn)
        times_custom.append(time_custom)
        times_sklearn.append(time_sklearn)
    
    all_times = np.array(times_custom + times_sklearn)
    bins = np.linspace(all_times.min(), all_times.max(), 11)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.bar(['Custom KMeans', 'Scikit-Learn KMeans'], [np.mean(times_custom), np.mean(times_sklearn)], color=['blue', 'red'])
    plt.ylabel('Average Execution Time (seconds)')
    plt.title('Comparison of Average Execution Time')

    plt.subplot(1, 2, 2)
    plt.hist(times_custom, bins=bins, alpha=0.7, label='Custom KMeans', color='blue')
    plt.hist(times_sklearn, bins=bins, alpha=0.7, label='Scikit-Learn KMeans', color='red')
    plt.xlabel('Execution Time (seconds)')
    plt.ylabel('Number of Runs')
    plt.title('Histogram of Execution Times')
    plt.legend()
    plt.tight_layout()

    plt.show()

test_stability(data)


On remarque que le temps d'execution de notre version sans numpy est beaucoup plus lente en comparaison avec la version de scikit-learn

# Tests de la version avec NumPy

In [ ]:
def calculate_metrics_NP(data, k_range):
    from kmeans_np import KMeansNP
    from sklearn.cluster import KMeans as SklearnKMeans
    from sklearn.metrics import silhouette_score
    
    sse_custom = {}
    sse_sklearn = {}
    silhouette_custom = {}
    silhouette_sklearn = {}
    
    for k in k_range:
        # Notre version de KMeans avec NumPy
        kmeans_custom = KMeansNP(k=k, max_iters=1000)
        _, centroids_custom = kmeans_custom.fit(data)
        sse_custom[k] = sum(min((sum((x - c) ** 2 for x, c in zip(point, centroid)) for centroid in centroids_custom)) for point in data)
        assignments_custom = [min(range(k), key=lambda i: sum((x - c) ** 2 for x, c in zip(point, centroids_custom[i]))) for point in data]
        silhouette_custom[k] = silhouette_score(data, assignments_custom)
        
        # Version scikit-learn de KMeans
        kmeans_sklearn = SklearnKMeans(n_clusters=k, init='k-means++',max_iter=1000, random_state=0)
        kmeans_sklearn.fit(data)
        sse_sklearn[k] = kmeans_sklearn.inertia_
        silhouette_sklearn[k] = silhouette_score(data, kmeans_sklearn.labels_)
    
    return sse_custom, sse_sklearn, silhouette_custom, silhouette_sklearn

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
X = digits.data
k_range = range(2, 10)
sse_custom, sse_sklearn, silhouette_custom, silhouette_sklearn = calculate_metrics_NP(X, k_range)


plot_metrics(sse_custom, sse_sklearn, silhouette_custom, silhouette_sklearn)

En terme de performances, on voit qu'on a rien perdu, testons maintenant le temps d'execution

In [ ]:
import time
import numpy as np
from sklearn.cluster import KMeans as SklearnKMeans
from kmeans_np import KMeansNP


def run_custom_kmeans(data):
    kmeans_custom = KMeansNP(k=4, max_iters=1000)
    start_time = time.time()
    assignments_custom, centroids_custom = kmeans_custom.fit(data)
    end_time = time.time()
    execution_time = end_time - start_time
    return centroids_custom, execution_time

def run_sklearn_kmeans(data):
    kmeans_sklearn = SklearnKMeans(n_clusters=4, init='k-means++',max_iter=1000, random_state=None)
    start_time = time.time()
    kmeans_sklearn.fit(data)
    end_time = time.time()
    execution_time = end_time - start_time
    return kmeans_sklearn.cluster_centers_, execution_time

def test_stability_NP(data, runs=10):
    results_custom = []
    results_sklearn = []
    times_custom = []
    times_sklearn = []
    
    for _ in range(runs):
        centroids_custom, time_custom = run_custom_kmeans(data)
        centroids_sklearn, time_sklearn = run_sklearn_kmeans(data)
        results_custom.append(centroids_custom)
        results_sklearn.append(centroids_sklearn)
        times_custom.append(time_custom)
        times_sklearn.append(time_sklearn)
    
    all_times = np.array(times_custom + times_sklearn)
    bins = np.linspace(all_times.min(), all_times.max(), 11)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.bar(['Custom KMeans', 'Scikit-Learn KMeans'], [np.mean(times_custom), np.mean(times_sklearn)], color=['blue', 'red'])
    plt.ylabel('Average Execution Time (seconds)')
    plt.title('Comparison of Average Execution Time')

    plt.subplot(1, 2, 2)
    plt.hist(times_custom, bins=bins, alpha=0.7, label='Custom KMeans', color='blue')
    plt.hist(times_sklearn, bins=bins, alpha=0.7, label='Scikit-Learn KMeans', color='red')
    plt.xlabel('Execution Time (seconds)')
    plt.ylabel('Number of Runs')
    plt.title('Histogram of Execution Times')
    plt.legend()
    plt.tight_layout()

    plt.show()

test_stability_NP(data)


On remarque qu'il y a un énorme gains en terme de temps d'execution sur le même jeu de données. Cela est dû principalement à l'utilisation de NumPy, il apporte des avantages significatifs en termes de vitesse grâce à la vectorisation, l'optimisation de la gestion de la mémoire, et les optimisations de bas niveau qui accélèrent les calculs.

Elle est aussi plus rapide que la version de scikit-learn. Cela peut etre dû au fait que scikit-learn est plus général et robuste pour une large variété de cas d'utilisation, ce qui peut introduire un surcoût en termes de performance.

Voyons voir maintenant jusqu'a combien de données cette version avec NumPy est plus rapide que la version de scikit-learn

In [ ]:
# Un ensemble de données sur des échantillons de vin avec des caractéristiques chimiques.
from sklearn.datasets import load_wine
wine = load_wine()
X1 = wine.data

k_range = range(2, 10)
sse_custom, sse_sklearn, silhouette_custom, silhouette_sklearn = calculate_metrics_NP(X, k_range)
plot_metrics(sse_custom, sse_sklearn, silhouette_custom, silhouette_sklearn)

test_stability_NP(X1)

Ici en voit que le temps d'execution n'est pas unanime, des fois c'est le notre qui est plus rapide, et d'autre fois c'est celui de scikit-learn. On commance à atteindre les limites où notre version est plus rapide. Mais en termes de performance de clustering, on reste sur des performances similaire entre les deux versions.

# Jeu de données encore plus complexe

In [ ]:
# Un ensemble de données d'images de chiffres manuscrits. Il s'agit de données en haute dimension (chaque image est de 8x8 pixels).
from sklearn.datasets import load_digits
digits = load_digits()
X2 = digits.data

k_range = range(2, 10)
sse_custom, sse_sklearn, silhouette_custom, silhouette_sklearn = calculate_metrics_NP(X, k_range)
plot_metrics(sse_custom, sse_sklearn, silhouette_custom, silhouette_sklearn)

test_stability_NP(X2)

Ici en voit que le temps d'execution est unanime, notre version est moins rapide que celle de scikit-learn. Mais en terme de performance de clustering, on reste sur des performances similaire entre les deux versions. Comme on l'a vu précédemment, scikit-learn est plus général et robuste pour une large variété de cas d'utilisation.

## Cas pratique sur un jeu de données réel

Nous allons maintenant tester sur un jeu de données réel tiré de le site `Kaggle`, il s'agit d'un jeu de données de clients d'un centre commercial. Le jeu de données contient des informations sur le genre, l'âge, le salaire annuel et le score de dépenses des clients. 

In [ ]:
import pandas as pd

df = pd.read_csv('Data/Mall_Customers.csv')
df.head()
df.isnull().sum()
X1 = df[['Age' , 'Spending Score (1-100)']].iloc[: , :].values

On commence par la version sans numpy

In [ ]:
kmeans_without_np= KMeans_Without_Numpy(k=4, max_iters=1000)
assignments_w_np, centroids_w_np = kmeans_without_np.fit(X1)

colors = ['r', 'g', 'b', 'm']  
plt.figure(figsize=(10, 6))

for idx, centroid in enumerate(centroids_w_np):
    cluster_points = [X1[i] for i in range(len(X1)) if assignments_w_np[i] == idx]
    cluster_points_x = [point[0] for point in cluster_points]
    cluster_points_y = [point[1] for point in cluster_points]
        
    plt.scatter(cluster_points_x, cluster_points_y, c=colors[idx], label=f'Cluster {idx}')
        
    plt.scatter(centroid[0], centroid[1], c=colors[idx], marker='x', s=200, linewidths=3)

plt.title('Visualisation des Clusters et des Centroids')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.legend()
plt.show()

Version avec numpy

In [ ]:
kmeans_np= KMeansNP(k=4, max_iters=1000)
assignments_np, centroids_np = kmeans_without_np.fit(X1)

colors = ['r', 'g', 'b', 'm']  
plt.figure(figsize=(10, 6))

for idx, centroid in enumerate(centroids_np):
    cluster_points = [X1[i] for i in range(len(X1)) if assignments_np[i] == idx]
    cluster_points_x = [point[0] for point in cluster_points]
    cluster_points_y = [point[1] for point in cluster_points]
        
    plt.scatter(cluster_points_x, cluster_points_y, c=colors[idx], label=f'Cluster {idx}')
        
    plt.scatter(centroid[0], centroid[1], c=colors[idx], marker='x', s=200, linewidths=3)

plt.title('Visualisation des Clusters et des Centroids')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.legend()
plt.show()

Maintenant à titre de comparaison, on va executer la version de scikit-learn

In [ ]:
from sklearn.cluster import KMeans

kmeans_sklearn = KMeans(n_clusters=4, init='k-means++')
kmeans_sklearn.fit(X1)

centroids_sklearn = kmeans_sklearn.cluster_centers_
labels_sklearn = kmeans_sklearn.labels_

colors = ['r', 'g', 'b', 'm','y']  
plt.figure(figsize=(10, 6))

centroids_sklearn = kmeans_sklearn.cluster_centers_
labels_sklearn = kmeans_sklearn.labels_

plt.figure(figsize=(10, 6))

for i in range(4):
    # Sélectionne les points de données du cluster i
    cluster_points = X1[labels_sklearn == i]
    # Trace les points de données
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], c=colors[i], label=f'Cluster {i}')
    # Trace le centroid
    plt.scatter(centroids_sklearn[i, 0], centroids_sklearn[i, 1], c=colors[i], marker='x', s=200, linewidths=3)

plt.title('Clustering avec scikit-learn KMeans')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.legend()
plt.show()

En comparant les trois résultats, on voit que les trois versions offrent des performances de clustering similaires. Cela confirme que notre implémentation de KMeans avec NumPy et sans sont capables de gérer des données réelles et de produire des résultats de clustering de qualité similaire à la version de scikit-learn.